## Commands using `sf` to inspect the scratch instance

In [7]:
!sf sobject list --sobject custom --target-org MyFirstScratch

No CUSTOM objects found.


In [16]:
!sf sobject describe --sobject Account --target-org MyFirstScratch | rg "HealthScore__c|ChurnRisk__c|Customer_Since__c|Segment__c|ARR__c|MRR__c|Support_Tier__c" || echo "no matches"

      "name": "ARR__c",
      "name": "ChurnRisk__c",
      "name": "Customer_Since__c",
      "name": "HealthScore__c",
      "name": "MRR__c",
      "name": "Segment__c",
      "name": "Support_Tier__c",


In [18]:
!sf org list

┌────┬─────────┬────────────────┬──────────┬──────────┬───────────┬────────────┐
│    │ Type    │ Alias          │ Username │ Org Id   │ Status    │ Expires    │
├────┼─────────┼────────────────┼──────────┼──────────┼───────────┼────────────┤
│ 🌳 │ DevHub  │ ac-dev-ed      │ benji30… │ 00DgL00… │ Connected │            │
│ 🍁 │ Scratch │ MyFirstScratch │ test-79… │ 00DAu00… │ Active    │ 2025-12-26 │
└────┴─────────┴────────────────┴──────────┴──────────┴───────────┴────────────┘


Legend:  🌳=Default DevHub, 🍁=Default Org      Use --all to see expired and deleted scratch orgs


In [17]:
!sf data query --query "SELECT AccountExtId__c FROM Account LIMIT 1" --target-org MyFirstScratch

Querying Data...
Querying Data... ⣾
Querying Data... ⣽
Querying Data... ⣻
Querying Data... ⢿
┌─────────────────┐
│ ACCOUNTEXTID__C │
├─────────────────┤
│ null            │
└─────────────────┘

Total number of records retrieved: 1.
Querying Data... done


## Duplicate Records Cleanup

This section handles the removal of duplicate records identified during testing.

**Objects with duplicates:**
- Product2: 40 actual (8 intended) → 32 duplicates
- Case: 150 actual (15 intended) → 135 duplicates
- Task: 225 actual (30 intended) → 195 duplicates
- FeedItem: 217 actual (20 intended) → 197 duplicates
- PricebookEntry: 98 actual (16 intended) → 82 duplicates

**Strategy:**
1. Backup all records to CSV (for audit trail)
2. Delete ALL records from these objects
3. Re-run loader to populate with correct counts
4. Verify final counts match CSV files

### Step 1: Verify Current Counts

In [1]:
import sys
from pathlib import Path
from datetime import datetime
import importlib

# Add parent directory to sys.path so we can import the package
parent_dir = Path("/Users/ac/dev/salesforce/ACDevHub/agents/python")
sys.path.insert(0, str(parent_dir))

# Import the package using importlib (handles hyphenated directory names)
restful_loaders = importlib.import_module('restful-loaders')
auth = importlib.import_module('restful-loaders.auth')
config = importlib.import_module('restful-loaders.config')
rest_client = importlib.import_module('restful-loaders.rest_client')

# Get the RestClient class
RestClient = rest_client.RestClient

# Authenticate with Salesforce
print("🔐 Authenticating with Salesforce...")
access_token, instance_url = auth.get_access_token()
rest = RestClient(access_token, instance_url)
print(f"✓ Connected to {instance_url}\n")

def get_count(rest_client, sobject):
    """Get count of records by querying all IDs and counting them."""
    try:
        # Query for IDs only (lightweight)
        result = rest_client.query(f"SELECT Id FROM {sobject}")
        # Count the number of records returned
        return len(result)
    except Exception as e:
        print(f"ERROR querying {sobject}: {e}")
        return None

# Get current counts
objects = ["Product2", "Case", "Task", "FeedItem", "PricebookEntry"]
print(f"Current counts as of {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}:\n")
print(f"{'Object':<20} {'Count':>10}")
print("-" * 30)

counts = {}
for obj in objects:
    count = get_count(rest, obj)
    if count is not None:
        counts[obj] = count
        print(f"{obj:<20} {count:>10}")
    else:
        print(f"{obj:<20} {'ERROR':>10}")

if counts:
    print(f"\nTotal records to delete: {sum(counts.values())}")
else:
    print("\n⚠️  No counts retrieved. Check connection and permissions.")

🔐 Authenticating with Salesforce...
✓ Connected to https://force-enterprise-762-dev-ed.scratch.my.salesforce.com

Current counts as of 2025-11-27 17:32:39:

Object                    Count
------------------------------
Product2                      8
Case                         15
Task                         30
FeedItem                     27
PricebookEntry               50

Total records to delete: 130


In [ ]:
# Debug: Let's see what a COUNT query actually returns
debug_result = rest.query("SELECT COUNT() FROM Account")
print("Debug - Raw query result for Account COUNT:")
print(f"Result: {debug_result}")
print(f"Type: {type(debug_result)}")
if debug_result:
    print(f"First item: {debug_result[0]}")
    print(f"Keys: {debug_result[0].keys()}")
    
# Also try a regular query to verify connection works
accounts = rest.query("SELECT Id, Name FROM Account LIMIT 3")
print(f"\nDebug - Regular query for Accounts:")
print(f"Found {len(accounts)} accounts")
if accounts:
    for acc in accounts[:3]:
        print(f"  - {acc.get('Name')}")

### Step 2: Backup Records to CSV (Audit Trail) ~~NO NEED~~

In [ ]:
import csv

# Create backup directory
backup_dir = Path("../../data/output/backup_before_deletion")
backup_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Backup queries - export all records for each object
backup_queries = {
    "Product2": "SELECT Id, Name, ProductCode, ProductExtId__c FROM Product2",
    "Case": "SELECT Id, Subject, Status, CaseNumber FROM Case",
    "Task": "SELECT Id, Subject, Status, WhatId, WhoId FROM Task",
    "FeedItem": "SELECT Id, Title, Type, ParentId FROM FeedItem",
    "PricebookEntry": "SELECT Id, Product2Id, Pricebook2Id, UnitPrice, IsActive FROM PricebookEntry"
}

print("Backing up records to CSV...\n")

for obj, query in backup_queries.items():
    output_file = backup_dir / f"{obj.lower()}_{timestamp}.csv"
    
    try:
        # Query using REST client
        records = rest.query(query)
        
        if not records:
            print(f"⚠️  {obj:<20} → No records to backup")
            continue
        
        # Write to CSV
        with output_file.open('w', newline='', encoding='utf-8') as f:
            # Get field names from first record (exclude 'attributes')
            fieldnames = [k for k in records[0].keys() if k != 'attributes']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            for record in records:
                # Remove 'attributes' metadata from record
                clean_record = {k: v for k, v in record.items() if k != 'attributes'}
                writer.writerow(clean_record)
        
        print(f"✓ {obj:<20} → {output_file.name} ({len(records)} records)")
    
    except Exception as e:
        print(f"❌ {obj:<20} → ERROR: {e}")

print(f"\nBackup completed! Files saved to: {backup_dir}")

### Step 3: Delete All Records (Dependency-Safe Order)

**Deletion order:**
1. FeedItem (child records)
2. Task (child records)
3. Case (child records)
4. PricebookEntry (references Product2)
5. Product2 (parent records)

In [ ]:
import time
import subprocess
import csv
from pathlib import Path

def delete_all_records(rest_client, sobject, display_name=None):
    """Delete all records from a Salesforce object using bulk API."""
    if display_name is None:
        display_name = sobject
    
    print(f"\n🗑️  Deleting all {display_name} records...")
    
    # Get count before deletion using REST client
    count_before = get_count(rest_client, sobject)
    print(f"   Records before: {count_before}")
    
    if count_before == 0:
        print(f"   ⚠️  No records to delete")
        return 0
    
    # Step 1: Query for all IDs
    print(f"   Querying for all IDs...")
    try:
        records = rest_client.query(f"SELECT Id FROM {sobject}")
        if not records:
            print(f"   ⚠️  No records found")
            return 0
    except Exception as e:
        print(f"   ❌ ERROR querying IDs: {e}")
        return 0
    
    # Step 2: Write IDs to CSV file with CRLF line endings (Salesforce Bulk API standard)
    temp_dir = Path("../../data/output/temp_delete")
    temp_dir.mkdir(parents=True, exist_ok=True)
    csv_file = temp_dir / f"{sobject.lower()}_delete.csv"
    
    print(f"   Writing {len(records)} IDs to CSV...")
    with csv_file.open('w', newline='') as f:
        # Salesforce Bulk API standard: CRLF (\r\n)
        # macOS default is LF (\n), so we must explicitly set CRLF
        writer = csv.writer(f, lineterminator='\r\n')
        writer.writerow(['Id'])  # Header
        for record in records:
            writer.writerow([record['Id']])
    
    # Step 3: Execute bulk delete using sf CLI with matching line-ending flag
    print(f"   Executing bulk delete...")
    result = subprocess.run(
        ["sf", "data", "delete", "bulk", 
         "--sobject", sobject,
         "--file", str(csv_file),
         "--line-ending", "CRLF",  # Must match CSV file line endings!
         "--wait", "10"],  # Wait up to 10 minutes
        capture_output=True,
        text=True
    )
    
    # Check if command failed
    if result.returncode != 0:
        print(f"   ❌ ERROR: {result.stderr[:300]}")
        print(f"   STDOUT: {result.stdout[:300]}")
        return 0
    
    # Wait a moment for deletion to propagate
    time.sleep(3)
    
    # Step 4: Get count after deletion
    count_after = get_count(rest_client, sobject)
    deleted = count_before - count_after
    
    print(f"   Records after: {count_after}")
    print(f"   ✓ Deleted: {deleted} records")
    
    # Clean up CSV file
    csv_file.unlink()
    
    if count_after > 0:
        print(f"   ⚠️  WARNING: {count_after} records remain (may be locked or have dependencies)")
    
    return deleted

# Delete in dependency-safe order
deletion_order = [
    ("FeedItem", "FeedItem"),
    ("Task", "Task"),
    ("Case", "Case"),
    ("PricebookEntry", "PricebookEntry"),
    ("Product2", "Product2")
]

print("=" * 60)
print("STARTING BULK DELETION")
print("=" * 60)

total_deleted = 0
for sobject, display_name in deletion_order:
    deleted = delete_all_records(rest, sobject, display_name)
    if deleted:
        total_deleted += deleted

print("\n" + "=" * 60)
print(f"DELETION COMPLETE - Total records deleted: {total_deleted}")
print("=" * 60)

### Step 4: Verify Final Counts

In [3]:
print(f"Final counts after deletion ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')}):\n")
print(f"{'Object':<20} {'Count':>10} {'Expected':>10} {'Status':>15}")
print("-" * 60)

all_zero = True
for obj in objects:
    count = get_count(rest, obj)
    if count is None:
        status = "❌ ERROR"
        all_zero = False
    elif count == 0:
        status = "✓ EMPTY"
    else:
        status = f"⚠️ {count} LEFT"
        all_zero = False
    
    print(f"{obj:<20} {count if count is not None else 'N/A':>10} {0:>10} {status:>15}")

print("-" * 60)
if all_zero:
    print("\n✅ SUCCESS: All objects are now empty!")
    print("Next step: Re-run the data loader to populate with correct counts")
    print("\nRun from project root:")
    print("  cd ../../")
    print("  uv run python -m restful-loaders.cli --verbose")
else:
    print("\n⚠️  WARNING: Some records remain. Check dependencies or locks.")
    print("You may need to manually delete remaining records or check for:")
    print("  - Record locks")
    print("  - Sharing rules")
    print("  - Workflow/trigger dependencies")

Final counts after deletion (2025-11-27 16:33:23):

Object                    Count   Expected          Status
------------------------------------------------------------
Product2                      0          0         ✓ EMPTY
Case                          0          0         ✓ EMPTY
Task                          0          0         ✓ EMPTY
FeedItem                      0          0         ✓ EMPTY
PricebookEntry               34          0      ⚠️ 34 LEFT
------------------------------------------------------------

⚠️  WARNING: Some records remain. Check dependencies or locks.
You may need to manually delete remaining records or check for:
  - Record locks
  - Sharing rules
  - Workflow/trigger dependencies


### Next Steps After Deletion

1. **Verify backup files exist:**
   ```bash
   ls -lh ../../data/output/backup_before_deletion/
   ```

2. **Re-run the data loader:**
   ```bash
   cd ../../
   uv run python -m restful-loaders.cli --verbose
   ```

3. **Verify final counts match CSV:**
   - Account: 8
   - Contact: 24
   - Product2: 8
   - Case: 15
   - Task: 30
   - FeedItem: 20
   - PricebookEntry: 16 (8 standard + 8 custom)
   - EmailMessage: 15
   - ContentDocument: 15